In [ ]:
# pip install transformers
# pip install datasets
# pip install lightning

In [ ]:
# Test imports one by one
try:
    import transformers
    print(f"✅ transformers: {transformers.__version__}")
except Exception as e:
    print(f"❌ transformers error: {e}")

try:
    from transformers import AutoModelForCausalLM, AutoTokenizer
    print("✅ AutoModel imports successful")
except Exception as e:
    print(f"❌ AutoModel error: {e}")

try:
    from transformers import Trainer
    print("✅ Trainer import successful")
except Exception as e:
    print(f"❌ Trainer error: {e}")

try:
    from transformers import BitsAndBytesConfig
    print("✅ BitsAndBytesConfig import successful")
except Exception as e:
    print(f"❌ BitsAndBytesConfig error: {e}")

In [ ]:
from transformers import AutoTokenizer

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token


def tokenize_example(example):
    prompt = f"""### Instruction:\n{example["instruction"]}\n\n### Response:\n{example["output"]}"""
    tokenized = tokenizer(prompt, truncation=True, padding="max_length", max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized


In [ ]:
from datasets import load_dataset, Dataset

# Load from Hugging Face Hub
raw_dataset = load_dataset("Josephgflowers/Finance-Instruct-500k", split="train")

# Inspect structure (one sample)
print(raw_dataset[0])


In [ ]:
def convert_to_instruction_format(example):
    return {
        "instruction": example["user"],
        "input": "",
        "output": example["assistant"]
    }

# Apply mapping
instruction_dataset = raw_dataset.map(convert_to_instruction_format)


In [ ]:
def tokenize(example):
    prompt = f"### Instruction:\n{example['instruction']}\n\n### Response:\n{example['output']}"
    tokenized = tokenizer(prompt, padding="max_length", truncation=True, max_length=512)
    tokenized["labels"] = tokenized["input_ids"].copy()
    return tokenized

tokenized_dataset = instruction_dataset.map(tokenize)


In [ ]:
# from transformers import AutoModelForCausalLM, BitsAndBytesConfig
# from peft import prepare_model_for_kbit_training, get_peft_model, LoraConfig

# bnb_config = BitsAndBytesConfig(
#     load_in_4bit=True,
#     bnb_4bit_quant_type="nf4",            # use efficient quant type
#     bnb_4bit_use_double_quant=True,       # double quantization
#     bnb_4bit_compute_dtype="float16"      # for better numeric stability
# )

# base_model = AutoModelForCausalLM.from_pretrained(
#     model_id,
#     quantization_config=bnb_config,
#     device_map="auto"
# )

# base_model = prepare_model_for_kbit_training(base_model)

# lora_config = LoraConfig(
#     r=16,
#     lora_alpha=32,
#     target_modules=["q_proj", "v_proj"],
#     lora_dropout=0.05,
#     bias="none",
#     task_type="CAUSAL_LM"
# )

# model = get_peft_model(base_model, lora_config)


In [ ]:
# from transformers import TrainingArguments, Trainer

# training_args = TrainingArguments(
#     output_dir="./finance_adapter",
#     per_device_train_batch_size=4,
#     gradient_accumulation_steps=4,
#     num_train_epochs=3,
#     learning_rate=2e-4,
#     logging_steps=10,
#     logging_dir="./finance_adapter/logs",   # <- logs directory
#     save_strategy="epoch",
#     fp16=True,
#     report_to="tensorboard",                # <- enable TensorBoard
# )


# trainer = Trainer(
#     model=model,
#     train_dataset=tokenized_dataset,
#     tokenizer=tokenizer,
#     args=training_args,
# )


In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import get_peft_model, LoraConfig

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

# Load model and tokenizer
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    torch_dtype="auto"
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# LoRA config (no quantization)
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=["q_proj", "v_proj"],
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(base_model, lora_config)


In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir="./finance_adapter",
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    num_train_epochs=3,
    learning_rate=2e-4,
    logging_steps=10,
    logging_dir="./finance_adapter/logs",
    save_strategy="epoch",
    fp16=True,  # Turn off if you're on CPU
    report_to="tensorboard",
)


In [ ]:
for name, param in model.named_parameters():
    if param.requires_grad:
        print("Trainable:", name)

In [ ]:
trainer.train()
model.save_pretrained("finance_adapter")


In [ ]:
%load_ext tensorboard
%tensorboard --logdir ./finance_adapter/logs


In [ ]:
from peft import PeftModel
base_model = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto"
)
model = PeftModel.from_pretrained(base_model, "finance_adapter")

prompt = "### Instruction:\nWhat is working capital?\n\n### Response:\n"
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to("cuda")

output_ids = model.generate(input_ids, max_new_tokens=100, do_sample=True, temperature=0.7)
print(tokenizer.decode(output_ids[0], skip_special_tokens=True))